### 목적
- LangChain 동적으로 시스템 프롬프트 변경 -> dynamic_prompt 미들웨어 사용

In [1]:
import os
from dotenv import load_dotenv

from langchain.agents import create_agent
from langchain.tools import tool
from langchain_openai import ChatOpenAI

from typing import TypedDict

from langchain.agents.middleware import dynamic_prompt, ModelRequest
# dynamic_prompt: 실행 시점에 프롬프트를 동적으로 수정하는 미들웨어 데코레이터 -> 모델 호출 전 
# ModelRequest: LLM에게 보낼 요청(프롬프트, 파라미터 등)을 담는 객체 -> LLM 호출 직전

load_dotenv()

True

In [3]:
llm = ChatOpenAI(
    model='gpt-4o-mini',
    api_key=os.getenv("OPENAI_API_KEY")
)

class Context(TypedDict):
    user_role: str

@dynamic_prompt
def user_role_prompt(request: ModelRequest) -> str:
    """
    사용자 역할에 따라 시스템 프롬프트를 생성하세요.
    """
    user_role = request.runtime.context.get('user_role', 'user')
    base_prompt = '너는 유용한 어시스턴트이다.'

    if user_role == 'expert':
        return f'{base_prompt} 자세한 기술적 답변을 제공하세요.'
    elif user_role == 'beginner':
        return f'{base_prompt} 개념을 간단하게 설명하고 전문 용어는 피하세요.'

    return base_prompt

agent = create_agent(
    model=llm,
    tools=[],
    middleware=[user_role_prompt],
    context_schema=Context
) 
# context_schema를 명시하여 내부 구조를 보장시킨다. 
# -> 넣지 않으면 None이나 키 에러 위험이 있음

result = agent.invoke(
    {'messages': [{'role': 'user', 'content': '머신러닝에 대해서 설명해라.'}]},
    context=Context(user_role='expert')
)

print(result.get('messages')[-1].content)

머신러닝(Machine Learning)은 인공지능(AI)의 한 분야로, 컴퓨터가 경험을 통해 자동으로 학습하고 개선할 수 있도록 하는 기술입니다. 주요 목표는 명시적으로 프로그래밍하지 않고도 데이터에서 패턴이나 규칙을 학습해 새로운 데이터에 대한 예측이나 결정을 내리는 것입니다.

### 머신러닝의 주요 유형

1. **지도학습(Supervised Learning)**:
   - 입력 데이터와 해당하는 출력 레이블이 주어지는 상황에서 모델을 학습시킵니다.
   - 주로 분류(Classification)와 회귀(Regression) 문제에 사용됩니다.
   - 예시: 스팸 이메일 탐지, 주택 가격 예측.

2. **비지도학습(Unsupervised Learning)**:
   - 출력 레이블이 없는 데이터에서 패턴이나 구조를 발견하도록 학습합니다.
   - 클러스터링(Clustering)과 차원 축소(Dimensionality Reduction)가 주요 기법입니다.
   - 예시: 고객 세분화, 데이터 시각화.

3. **강화학습(Reinforcement Learning)**:
   - 에이전트가 환경과 상호작용하며 보상을 극대화하기 위해 행동을 학습합니다.
   - 주로 게임, 로봇 공학, 자율주행차 등에 적용됩니다.
   - 예시: 알파고(AlphaGo), 자율주행 차량 제어.

### 머신러닝 프로세스

1. **데이터 수집**: 문제 해결에 필요한 데이터를 수집합니다.
2. **데이터 전처리**: 데이터 정제 및 변환 작업을 수행하여 모델 학습에 적합한 형태로 만듭니다.
3. **특징 선택 및 추출**: 모델이 학습하는 데 도움이 되는 유의미한 특징(feature)을 선택하거나 생성합니다.
4. **모델 선택**: 주어진 문제에 적합한 머신러닝 알고리즘을 선택합니다.
5. **모델 학습**: 선택된 알고리즘을 사용하여 데이터를 통해 모델을 학습시킵니다.
6. **모델 평가**: 검증 데이터셋을 이용해 모델의 성능을 평가합니다.
7. **모델 튜닝**: 

#### 미들웨어 총 정리
- dynamic_prompt: context, 사용자 상태, 시간 등에 따라 system prompt 수정
- wrap_model_call: LLM에게 보낼 요청 또는 응답을 가로채서 수정/검증/단축 종료
- wrap_tool_call: 에이전트가 선택한 툴 호출의 입력/출력을 감싸서 처리

#### 실행 순서
1. 사용자 입력
2. @dynamic_prompt -> 시스템 프롬프트 생성
3. @wrap_model_call -> LLM 호출 전후 개입 (prompt, 모델 교체 등)
4. LLM 판단 (어떤 툴 쓸지 결정)
5. @wrap_tool_call -> 선택된 툴 실행 전후 개입
6. 툴 결과 Observation 생성
7. @wrap_model_call -> 다시 LLM 호출 (결과 요약 또는 최종 응답 생성)
8. 최종 출력